In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
from collections import Counter
warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

In [2]:
df_goudi1_bkp = pd.read_excel('../../Data/CompInfo/购地-地块公示.xlsx')
df_goudi2_bkp = pd.read_excel('../../Data/CompInfo/购地-市场交易-土地转让.xlsx')
df_goudi3_bkp = pd.read_excel('../../Data/CompInfo/购地-房地产大企业购地情况.xlsx')
df_goudi4_bkp = pd.read_excel('../../Data/CompInfo/购地-房地产大地块出让情况.xlsx')
df_goudi5_bkp = pd.read_excel('../../Data/CompInfo/购地-市场交易-土地抵押.xlsx')
df_goudi6_bkp = pd.read_excel('../../Data/CompInfo/购地-结果公告.xlsx')
df_goudi1 = df_goudi1_bkp.copy()
df_goudi2 = df_goudi2_bkp.copy()
df_goudi3 = df_goudi3_bkp.copy()
df_goudi4 = df_goudi4_bkp.copy()
df_goudi5 = df_goudi5_bkp.copy()
df_goudi6 = df_goudi6_bkp.copy()
print(df_goudi1.info(), '\n地块公示')
print('=================')
print(df_goudi3.info(), '\n房地产大企业购地情况')
print('=================')
print(df_goudi2.info(), '\n市场交易-土地转让')
print('=================')
print(df_goudi4.info(), '\n房地产大地块出让情况')
print('=================')
print(df_goudi5.info(), '\n市场交易-土地抵押')
print('=================')
print(df_goudi6.info(), '\n结果公告')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20033 entries, 0 to 20032
Data columns (total 9 columns):
企业编号        20033 non-null int64
行政区         19993 non-null object
时间          20033 non-null object
公示日期        5046 non-null object
土地用途        20027 non-null object
土地面积（公顷）    20033 non-null float64
出让年限        3970 non-null object
成交价（万元）     4894 non-null float64
土地使用条件      2032 non-null object
dtypes: float64(2), int64(1), object(6)
memory usage: 1.4+ MB
None 
地块公示
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20033 entries, 0 to 20032
Data columns (total 11 columns):
企业编号         20033 non-null int64
行政区          19993 non-null object
签订日期         20033 non-null object
供地总面积（公顷）    20033 non-null float64
约定动工时间       1084 non-null object
土地用途         20027 non-null object
供应方式         1032 non-null object
最小容积率        513 non-null float64
最大容积率        517 non-null float64
成交价款（万元）     4894 non-null float64
约定竣工时间       631 non-null object
dtypes: float64(4), int64(1)

In [3]:
print('地块公示 vs 房地产大企业购地情况:', df_goudi1['土地面积（公顷）'].equals(
    df_goudi3['供地总面积（公顷）']), df_goudi1['企业编号'].equals(
    df_goudi3['企业编号']))
print('市场交易-土地转让 vs 房地产大地块出让情况:',
      df_goudi2['土地面积(公顷)'].equals(df_goudi4['供地总面积']), df_goudi2['企业编号'].equals(df_goudi4['企业编号']))
print('市场交易-土地转让 vs 市场交易-土地抵押:',
      df_goudi2['土地面积(公顷)'].equals(df_goudi5['抵押面积(公顷)']), df_goudi2['企业编号'].equals(df_goudi5['企业编号']))
print('市场交易-土地转让 vs 结果公告:', df_goudi2['土地面积(公顷)'].equals(
    df_goudi6['总面积']), df_goudi2['企业编号'].equals(df_goudi6['企业编号']))

地块公示 vs 房地产大企业购地情况: True True
市场交易-土地转让 vs 房地产大地块出让情况: True True
市场交易-土地转让 vs 市场交易-土地抵押: False True
市场交易-土地转让 vs 结果公告: True True


In [7]:
# df_goudi_gongshi_qingkuai = df_goudi1.merge(df_goudi3, on='企业编号', how='outer')
# df_goudi24 = df_goudi2.merge(df_goudi4, on='企业编号', how='outer')
# df_goudi_merge = df_goudi24.merge(df_goudi6, on='企业编号', how='outer')

In [9]:
df_goudi1['土地面积（公顷）'] = df_goudi1['土地面积（公顷）'].astype(float)
goudi_grouped = df_goudi1.groupby(['企业编号'])

agg_goudi_func = {
    '土地面积（公顷）': ['sum', 'min', 'max']
}
goudi_grouped_info = goudi_grouped.agg(agg_goudi_func)
goudi_grouped_info.columns = [
    '_'.join(col).strip() for col in goudi_grouped_info.columns.values]
goudi_grouped_info['企业编号'] = goudi_grouped_info.index
goudi_grouped_info = goudi_grouped_info.set_index(
    np.arange(0, len(goudi_grouped_info)))
goudi_grouped_info.head(n=3)
# drop goudi3

,土地面积（公顷）_sum,土地面积（公顷）_min,土地面积（公顷）_max,企业编号
0,158135.998846,0.00030,80001.80000,1001
1,55.668100,4.15927,41.40896,1003
2,4.000000,0.60000,3.40000,1004


In [28]:
df_jine = df_goudi5.loc[(df_goudi5['评估金额(万元)'].notnull())]

df_jine['评估金额(亿元)'] = df_jine['评估金额(万元)'].astype(float)/10000
goudi_grouped_jine = df_jine.groupby(['企业编号'])

agg_goudi_func = {
    '评估金额(亿元)': ['sum', 'max']
}
goudi_grouped_jine_info = goudi_grouped_jine.agg(agg_goudi_func)
goudi_grouped_jine_info.columns = [
    '_'.join(col).strip() for col in goudi_grouped_jine_info.columns.values]
goudi_grouped_jine_info['企业编号'] = goudi_grouped_jine_info.index
goudi_grouped_jine_info = goudi_grouped_jine_info.set_index(
    np.arange(0, len(goudi_grouped_jine_info)))
goudi_grouped_jine_info.head(n=3)

,评估金额(亿元)_sum,评估金额(亿元)_max,企业编号
0,54454.534553,16793.953500,1001
1,1072.827287,304.009500,1010
2,1831.685186,40.403526,1011


In [50]:
merge_goudi = goudi_grouped_info.merge(
    goudi_grouped_jine_info, how='outer', on='企业编号').fillna(0)
merge_goudi['土地面积评估均值(公顷/亿元)'] = (merge_goudi['土地面积(公顷)_sum'] /
                                  merge_goudi['评估金额(亿元)_sum']).replace(np.inf, 0)
cols = ['企业编号', '土地面积(公顷)_sum', '土地面积(公顷)_min', '土地面积(公顷)_max',
        '评估金额(亿元)_sum', '评估金额(亿元)_max', '土地面积评估均值(公顷/亿元)']
merge_goudi = merge_goudi[cols]

diyacishu = pd.DataFrame()
diyacishu['抵押次数'] = df_goudi5.loc[(df_goudi5['评估金额(万元)'].notnull())].groupby(['企业编号'])[
    '企业编号'].count()

goudicishu = pd.DataFrame()
goudicishu['购地次数'] = df_goudi1.loc[(df_goudi1['土地面积（公顷）'].notnull())].groupby(['企业编号'])[
    '企业编号'].count()
goudicishu = goudicishu.merge(
    diyacishu, how='outer', on='企业编号').fillna(0).astype(int)
merge_goudi = merge_goudi.merge(goudicishu, how='outer', on='企业编号').fillna(0)
merge_goudi.head(n=20)

,企业编号,土地面积(公顷)_sum,土地面积(公顷)_min,土地面积(公顷)_max,评估金额(亿元)_sum,评估金额(亿元)_max,土地面积评估均值(公顷/亿元),购地次数,抵押次数
0,1001,158135.998846,0.000300,80001.80000,54454.534553,16793.953500,2.904001,1671,1528
1,1003,55.668100,4.159270,41.40896,0.000000,0.000000,0.000000,3,0
2,1004,4.000000,0.600000,3.40000,0.000000,0.000000,0.000000,2,0
3,1005,2.935250,2.935250,2.93525,0.000000,0.000000,0.000000,1,0
4,1007,230.110791,30.492800,66.66667,0.000000,0.000000,0.000000,4,0
5,1008,232.835300,0.004700,50.35500,0.000000,0.000000,0.000000,18,0
6,1009,63.385300,0.533500,32.47550,0.000000,0.000000,0.000000,8,0
7,1010,66387.293000,0.001100,38089.20000,1072.827287,304.009500,61.880690,773,753
8,1011,5866.556554,0.000200,180.01310,1831.685186,40.403526,3.202819,1292,1245
9,1012,361.606180,1.740800,179.06229,0.000000,0.000000,0.000000,4,0
